<a href="https://colab.research.google.com/github/ucfilho/SwarmCantStopNow/blob/main/Conference_2021/ABCOptmi_from_C_language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from random import randint

In [ ]:
def CalcFit(fun):
    result=0
    if(fun>=0):
      result=1/(fun+1)
    else:
      result=1+fabs(fun)
    return result

In [ ]:
def function( sol):
  #Schwefel(x):
  x=sol  
  summ=0
  for i in range(len(x)):
    new=x[i]*np.sin((abs(x[i]))**0.5)
    summ=summ+new
    top=(418.9829*len(x)-summ) 
  
  return top;


In [ ]:
def BestSource(GlobMin,GlobPars,Foods):
  FoodNumber=len(Foods[:,0])
  D=len(Foods[0,:])
  solution=np.zeros(D)
  for i in range(FoodNumber):
    if (f[i]<GlobMin):
      GlobMin=f[i]
      for j in range(D):
        GlobPars[j]=Foods[i,j]
  return GlobMin,GlobPars,Foods

In [ ]:
def init(index,Foods,trial,f,fitness,MIN,MAX):
  D=len(Foods[0,:])
  FoodNumber=len(Foods[:,0])
  solution=np.zeros(D)
  for j in range(D):
    r=np.random.random()
    Foods[index,j]=r*(MAX[j]-MIN[j])+MIN[j]
    solution[j]=Foods[index,j]
    
  f[index]=function(solution)
  fitness[index]=CalcFit(f[index])
  trial[index]=0
  
  return Foods,trial,f,fitness

In [ ]:
def initial(fitness,trial,f,Foods,GlobMin,GlobPars,MIN,MAX):
  D=len(Foods[0,:])
  FoodNumber=len(Foods[:,0])
  for i in range(FoodNumber):
    Foods,trial,f,fitness=init(i,Foods,trial,f,fitness,MIN,MAX) 
  GlobMin=f[0]
  for i in range(D):
    GlobPars[i]=Foods[0,i]
  return  f,Foods,GlobMin,GlobPars


In [ ]:
def EmployedBees(trial,Foods,MIN,MAX):
  FoodNumber=len(Foods[:,0])
  NP=FoodNumber
  D=len(Foods[0,:])
  solution=np.zeros(D)
  for i in range(FoodNumber):
    r = np.random.random()
    par2chan= int(r*D)
    r = np.random.random()
    neighbour=int(r*D)
    if(neighbour >= NP):
      neighbour=NP-1
      
    while(neighbour==i):
      r = np.random.random()
      neighbour=int(r*FoodNumber)
      if(neighbour >= NP):
        neighbour=NP-1
    for j in range(D):
      solution[j]=Foods[i,j]

    r = np.random.random()
    A=Foods[i,par2chan]
    B=Foods[neighbour,par2chan]
    solution[par2chan]=Foods[i,par2chan]+(A-B)*(r-0.5)*2;
    if (solution[par2chan]<MIN[par2chan]):
      solution[par2chan]=MIN[par2chan]
    if (solution[par2chan]>MAX[par2chan]):
      solution[par2chan]=MAX[par2chan]
    ObjValSol=function(solution)
    FitnessSol=CalcFit(ObjValSol)
    
    if(FitnessSol>fitness[i]):
      trial[i]=0
      for j in range(D):
        Foods[i,j]=solution[j]
      f[i]=ObjValSol
      fitness[i]=FitnessSol
    else:
      trial[i]=trial[i]+1 
      
  return trial,Foods

In [ ]:
def CalcProb(fitness,prob):
  FoodNumber=len(fitness)
  maxfit=fitness[0]
  for i in range(1,FoodNumber):    
    if (fitness[i]>maxfit):
      maxfit=fitness[i]
  for i in range(FoodNumber):
    prob[i]=(0.9*(fitness[i]/maxfit))+0.1
  return fitness,prob


In [ ]:
def OnlookerBees(trial,Foods,MIN,MAX):
  D=len(Foods[0,:])
  solution=np.zeros(D)
  FoodNumber=len(Foods[:,0])
  i=0
  t=0
  while(t<FoodNumber):
    r = np.random.random()
    if(r<prob[i]): 
      r = np.random.random()
      t=t+1
      par2chan=int(r*D)
      r = np.random.random()
      neighbour=int(r*FoodNumber)

      while(neighbour==i):
        r = np.random.random()
        neighbour=int(r*FoodNumber)
      for j in range(D):
        solution[j]=Foods[i,j]

      r = np.random.random()
      A=Foods[i,par2chan]
      B=Foods[neighbour,par2chan]
      solution[par2chan]=A+(A-B)*(r-0.5)*2

      if (solution[par2chan]<MIN[par2chan]):
              solution[par2chan]=MIN[par2chan]
      if (solution[par2chan]>MAX[par2chan]):
              solution[par2chan]=MAX[par2chan]
      ObjValSol=function(solution)
      FitnessSol=CalcFit(ObjValSol)
              
      if(FitnessSol>fitness[i]):
        trial[i]=0
        for j in range(D):
          Foods[i,j]=solution[j]
        f[i]=ObjValSol
        fitness[i]=FitnessSol
      else:
        trial[i]=trial[i]+1
      i=i+1
      if (i==FoodNumber):
        i=0
  return trial,Foods


In [ ]:
def ScoutBees(fitness,f,Foods,trial,MIN,MAX):
  FoodNumber=len(Foods[:,0])
  max_trial=0
  for i in range(1,FoodNumber):
    if (trial[i]>trial[max_trial]):
      max_trial=i
  if(trial[max_trial]>=limit):
    Foods,trial,f,fitness=init(max_trial,Foods,trial,f,fitness,MIN,MAX)
  return trial,Foods


In [ ]:
'''/*Main program of the ABC algorithm*/'''

NPAR=30 #Numero de fontes de comida
ITE=50 #ITERACOES (maxCycle)
PAR=3 #NUM DE PARAMETROS A SER OTIMIZADOS
MAX=[500,500,500] # MAXIMO DE CADA PARAMETRO
MIN=[-500,-500,-500] # MINIMO DE CADA PARAMETRO

limit=100 # quantas vezes obtem resposta identica antes de encerrar 
runtime=4 # quantas vezes vai rodar para tirar a media

f=np.zeros(NPAR) 
Foods=np.zeros((NPAR,PAR)) 
solution=np.zeros(PAR)
fitness=np.zeros(NPAR)
trial=np.zeros(NPAR)
prob=np.zeros(NPAR)
GlobPars=np.zeros(PAR)
GlobMins=np.zeros(runtime)
GlobMin=1e99

for run in range(runtime):
  f,Foods,GlobMin,GlobPars= initial(fitness,trial,f,Foods,GlobMin,GlobPars,MIN,MAX)
  GlobMin,GlobPars,Foods=BestSource(GlobMin,GlobPars,Foods)

  for iter in range(ITE):
    trial,Foods=EmployedBees(trial,Foods,MIN,MAX)
    fitness,prob=CalcProb(fitness,prob)
    trial,Foods=OnlookerBees(trial,Foods,MIN,MAX)
    GlobMin,GlobPars,Foods=BestSource(GlobMin,GlobPars,Foods)
    trial,Foods=ScoutBees(fitness,f,Foods,trial,MIN,MAX)

  for j in range(PAR):
    print("GlobalParam[",j+1,"]:", GlobPars[j])
    
  print("run",run+1,"GBest=",GlobMin," \n");
  GlobMins[run]=GlobMin

mean=np.average(GlobMins)


print("Means of",runtime,"runs:",mean,"\n")

GlobalParam[ 1 ]: 420.9683247750513
GlobalParam[ 2 ]: 420.98318576359696
GlobalParam[ 3 ]: 420.9607740832281
run 1 GBest= 7.253392777784029e-05  

GlobalParam[ 1 ]: 419.8968806732051
GlobalParam[ 2 ]: -302.56108923776634
GlobalParam[ 3 ]: 421.57851253599114
run 2 GBest= 118.63036895680489  

GlobalParam[ 1 ]: 420.61389216597024
GlobalParam[ 2 ]: 420.9767851694182
GlobalParam[ 3 ]: 419.96152054599946
run 3 GBest= 0.1438908143363733  

GlobalParam[ 1 ]: 420.8869078223751
GlobalParam[ 2 ]: 422.1153690808895
GlobalParam[ 3 ]: 420.2675548317038
run 4 GBest= 0.22886683281444675  

Means of 4 runs: 29.75079978447087 



In [ ]:
'''
ITE=300 #ITERACOES (maxCycle)


GlobalParam[ 1 ]: 420.96874614718286
GlobalParam[ 2 ]: 420.9687461719512
GlobalParam[ 3 ]: 420.9687463768404
run 1 GBest= 3.818269851763034e-05  

GlobalParam[ 1 ]: 420.9687462577133
GlobalParam[ 2 ]: 420.9687461387745
GlobalParam[ 3 ]: 420.9687457381019
run 2 GBest= 3.818269851763034e-05  

GlobalParam[ 1 ]: 420.9687462047169
GlobalParam[ 2 ]: 420.9687461691041
GlobalParam[ 3 ]: 420.96874633691976
run 3 GBest= 3.818269851763034e-05  

GlobalParam[ 1 ]: 420.9687464772267
GlobalParam[ 2 ]: 420.96874616214797
GlobalParam[ 3 ]: 420.9687461513788
run 4 GBest= 3.818269851763034e-05  

Means of 4 runs: 3.818269851763034e-05 
'''

'\nITE=300 #ITERACOES (maxCycle)\n\n\nGlobalParam[ 1 ]: 420.96874614718286\nGlobalParam[ 2 ]: 420.9687461719512\nGlobalParam[ 3 ]: 420.9687463768404\nrun 1 GBest= 3.818269851763034e-05  \n\nGlobalParam[ 1 ]: 420.9687462577133\nGlobalParam[ 2 ]: 420.9687461387745\nGlobalParam[ 3 ]: 420.9687457381019\nrun 2 GBest= 3.818269851763034e-05  \n\nGlobalParam[ 1 ]: 420.9687462047169\nGlobalParam[ 2 ]: 420.9687461691041\nGlobalParam[ 3 ]: 420.96874633691976\nrun 3 GBest= 3.818269851763034e-05  \n\nGlobalParam[ 1 ]: 420.9687464772267\nGlobalParam[ 2 ]: 420.96874616214797\nGlobalParam[ 3 ]: 420.9687461513788\nrun 4 GBest= 3.818269851763034e-05  \n\nMeans of 4 runs: 3.818269851763034e-05 \n'